In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Created on Aug 25 14:47:58 2023

@author: mariapopovic
"""

from numpy import ceil, sqrt, gradient, arange, e, array, exp, append, linspace, log, average, around, arange, sqrt, transpose, random, inf, multiply, linalg, matmul, histogram
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from math import erf, pi, erfc, log2
from scipy.special import erfinv
from scipy.integrate import quad
from scipy.stats import norm
from statistics import mean, stdev, variance
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Quantizers Class

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri May 10 17:23:15 2024

@author: mariapopovic
"""


from numpy import pi, sqrt, random, exp, append, log2, zeros
from numpy import inf
from scipy.optimize import fsolve
from math import erf, erfc
from scipy.stats import norm
from scipy.integrate import quad


class optimal_quantizers(object):
    
    def __init__(self, bins):
        
        self.mu = 0
        self.sigma = 1
        
        self.number_bins_list = [bins]

        ##### LINEAR KNOTS #####
        self.probabilities_linear = [[]]*len(self.number_bins_list)
        self.knots_linear = [] 
        self.cdf_linear_rightbound = [[]]*len(self.number_bins_list)
        self.cell_bounds_linear = [[]]*len(self.number_bins_list)
        self.y_linear = [] 
        self.alpha_list = []

        ##### STATIONARY KNOTS #####
        self.probabilities_stationary = [[]]*len(self.number_bins_list)
        self.knots_stationary= [] 
        self.cdf_stationary_rightbound = [[]]*len(self.number_bins_list)
        self.cell_bounds_stationary = [[]]*len(self.number_bins_list)
        self.y_stationary = []
        

    ############################# STATIONARY ###########################################
    # Define stationary knots and their probabilitites
    
    def create_stationary_knots(self):

        def standard_normal(x, mu, sigma):
    
            fx =  (1/(sqrt(2*pi)*sigma))*exp(-(1/2)*(((x - mu)/sigma)**2))
    
            return fx
            
        for index_bins in range(0,len(self.number_bins_list)):

            n = int(self.number_bins_list[index_bins]/2)

            def equations_knots_stationary(gamma):

                prob = zeros(n)
                Eq = zeros(n)

                if n == 1:
                    prob[0] = 1/2
                    Eq[0] = gamma[0] - (1/(sqrt(2*pi)*prob[0]))

                else:
                    prob[0] = 0.5*erf((gamma[0] + gamma[1])/(2*sqrt(2)))
                
                    for jj in range(1, n - 1):
                        prob[jj] = 0.5*(erf((gamma[jj] + gamma[jj + 1])/(2*sqrt(2))) - erf((gamma[jj - 1] + gamma[jj])/(2*sqrt(2))))
                    
                    prob[n - 1] = 0.5*(1 - erf((gamma[n - 1] + gamma[n -2])/(2*sqrt(2))))

                    Eq[0] = gamma[0] - (( 1 - exp( - 0.5*( ( (gamma[0] + gamma[1])*0.5 )**2 ) ) )/(sqrt(2*pi)*prob[0]))
                
                    for jj in range(1, n - 1):
                        Eq[jj] = gamma[jj] - (( exp( - 0.5*( ( (gamma[jj] + gamma[jj - 1])*0.5 )**2 ) ) - exp( - 0.5*( ( (gamma[jj] + gamma[jj + 1])*0.5 )**2 ) ) )/(sqrt(2*pi)*prob[jj]))

                    Eq[n - 1] = gamma[n - 1] - ( exp( - 0.5*( ( (gamma[n - 1] + gamma[n - 2])*0.5 )**2 ) ) /(sqrt(2*pi)*prob[n - 1]))
                #p_i
                return Eq

            start_points = []
          
            j = 1
            while j <= n:
                start_points.append(3*j/n)
                j = j + 1

            gamma = fsolve(equations_knots_stationary, start_points)
            gammas_stationary = append(- gamma[::-1], gamma)
            
            # If mean is not 0 and variance is not 1
            for index in range(0, len(gammas_stationary)):
                
                gammas_stationary[index] =  gammas_stationary[index]*self.sigma + self.mu

            self.knots_stationary.append(gammas_stationary)
        
        # Probabilities
            def probabilities_stat(gamma):
                
                prob = zeros(n)

                if n == 1:
                    prob[0] = 1/2
                    
                else:
                    
                    prob[0] = 0.5*erf((gamma[0] + gamma[1])/(2*sqrt(2)))
                
                    for jj in range(1, n - 1):
                        prob[jj] = 0.5*(erf((gamma[jj] + gamma[jj + 1])/(2*sqrt(2))) - erf((gamma[jj - 1] + gamma[jj])/(2*sqrt(2))))
                    
                    prob[n - 1] = 0.5*(1 - erf((gamma[n - 1] + gamma[n -2])/(2*sqrt(2))))

                return prob

            length = len(gammas_stationary)
            middle = length // 2
            tmp_probs = probabilities_stat(gammas_stationary[middle:])
            self.probabilities_stationary[index_bins] = append(tmp_probs[::-1], tmp_probs)

            self.cell_bounds_stationary[index_bins] = [-inf]
            
            for index2 in range(1, len(self.knots_stationary[index_bins])):
                self.cell_bounds_stationary[index_bins].append( (gammas_stationary[index2] + gammas_stationary[index2 - 1])/2 )
                
            self.cell_bounds_stationary[index_bins].append(inf)

            # Define the cumulative distribution function from the right extreme of each cell

            self.cdf_stationary_rightbound[index_bins] = append(self.cdf_stationary_rightbound[index_bins],\
                                                    norm.cdf(self.cell_bounds_stationary[index_bins][1:], loc = self.mu, scale = self.sigma) )

            # Round last element to = 1
    
            tmp = self.cdf_stationary_rightbound[index_bins][:-1]
            self.cdf_stationary_rightbound[index_bins] = append(tmp, round(self.cdf_stationary_rightbound[index_bins][-1]))

        # Extract one linear quantizer according to their probability distribution
    
    def stationary_quantizer(self):
        
        u = random.uniform(0, 1)
        
        for index_bins in range(0, len(self.number_bins_list)):
        
            index2 = 0
    
            while self.cdf_stationary_rightbound[index_bins][index2] < u:
                index2 = index2 + 1

            self.y_stationary.append(self.knots_stationary[index_bins][index2])
        
    ############################# LINEAR ###########################################

    # Define linear knots and their probabilitites
    def create_linear_knots(self):
           
        for index_bin in range(0,len(self.number_bins_list)):
    
            n = self.number_bins_list[index_bin]/2
            
            def standard_normal(x, mu, sigma):
    
                fx =  (1/(sqrt(2*pi)*sigma))*exp(-(1/2)*(((x - mu)/sigma)**2))
    
                return fx

            def first_integral(alpha, N):

                fsum = 0

                for j in range(1, N):
        
                    b0 = (alpha/N)*(j - 1)
                    b1 = (alpha/N)*j
                    fsum += quad(lambda x: ( (alpha/N)*((j - 0.5)**2) - (j - 0.5)*x )*standard_normal(x, 0, 1), b0, b1)[0]
    
                return fsum

            def second_integral(alpha, N):
    
                b0 = alpha - (alpha/N)
                b1 = inf
    
                return quad(lambda x: (N - 0.5)*( alpha*(1 - (1/(2*N))) - x)*standard_normal(x, 0, 1), b0, b1)[0]

            def equations_optimal_linear(alpha):
                
                return first_integral(alpha, int(n)) + second_integral(alpha, int(n))


            a = fsolve(equations_optimal_linear, 1)[0]
            self.alpha_list.append(a)
    
            start = -a + (a/(2*n)) # for even quantizers
            h = a/n

            tmp = []
            for i in range(1, int(2*n) + 1):
                gg = h*(i - 1) + start
                tmp.append(gg)
  
            
            # If mean is not 0 and variance is not 1
            for index in range(0, len(tmp)):
                
                tmp[index] =  tmp[index]*self.sigma + self.mu

            self.knots_linear.append(tmp)
            
            # Cell bounds
            
            self.cell_bounds_linear[index_bin] = append(self.cell_bounds_linear[index_bin], -inf)

            for i in range(1, int(2*n)):
                bound = h*(i - n)
                self.cell_bounds_linear[index_bin] = append(self.cell_bounds_linear[index_bin], self.sigma*bound + self.mu)
                self.cdf_linear_rightbound[index_bin] = append(self.cdf_linear_rightbound[index_bin],\
                                                               norm.cdf(self.cell_bounds_linear[index_bin][-1], loc = self.mu, scale = self.sigma))

            # Last bin

            self.cell_bounds_linear[index_bin] = append(self.cell_bounds_linear[index_bin], inf)
            self.cdf_linear_rightbound[index_bin] = append(self.cdf_linear_rightbound[index_bin],\
                                                           norm.cdf(self.cell_bounds_linear[index_bin][-1], loc = self.mu, scale = self.sigma))

    # Create probabilities

        for index_bins in range(0,len(self.number_bins_list)):
            
            n = int(self.number_bins_list[index_bins]/2)
            tmp = []
            al = self.alpha_list[index_bins]
            
            for j in range(1, n + 1):
                
                b0 = (al/n)*(j - 1)
                b1 = (al/n)*(j)
                new_prob = quad(lambda x: standard_normal(x, 0, 1), b0, b1)[0]
                tmp.append(new_prob)

            
            self.probabilities_linear[index_bins] = tmp
            self.probabilities_linear[index_bins] = append(tmp[::-1], self.probabilities_linear[index_bins])
            
    
    # Extract one linear quantizer according to their probability distribution
    def linear_quantizer(self):
        
        u = random.uniform(0, 1)
        
        for index_bins in range(0,len(self.number_bins_list)):
        
            index2 = 0
    
            while self.cdf_linear_rightbound[index_bins][index2] < u:
                index2 = index2 + 1

            self.y_linear.append(self.knots_linear[index_bins][index2])
            
            

In [3]:
def linear_quantizer(p, knotsZero, cell_boundsZero):

    shifted_knots = []
    
    for index in range(0, len(knotsZero[0])):
                
        shifted_knots.append((knotsZero[0])[index]*p[1] + p[0])

    
    return shifted_knots


def stationary_quantizer(p, knotsZero, cell_boundsZero):

    shifted_knots = []
    
    for index in range(0, len(knotsZero)):
                
        shifted_knots.append(knotsZero[index]*p[1] + p[0])

    
    return shifted_knots[0]

# Dual grid

In [4]:
from numpy import random

# Create probability list
def prob(p0, sigma, N):
    
    mu = log(p0) - ((sigma**2)/2)
    
    p = []
    
    for _ in range(N):
    
        r = random.lognormal(mu, sigma**2)
        p.append(r)

    
    return p


# Get dual knots
def dual_grid(alpha_list, N_list, prob_list, k, knots_lin, cell_bounds, optimized):
    
    N_k = N_list[k - 1] 
    
    
    mu_k = 0
    sigma_k = 0
    for jj in range(0, k):
        a_jj = alpha_list[jj]
        p_jj = prob_list[jj]
        mu_k = mu_k + a_jj*p_jj
        sigma_k = sigma_k + (a_jj**2)*p_jj*(1 - p_jj)
    
    # Normal codebook
    
    point_k = [mu_k, sigma_k]
    
    knots_k = linear_quantizer(point_k, knots_lin, cell_bounds)

    upper_bound = 0
    for jj in range(0, k):
        upper_bound = upper_bound + alpha_list[jj]
        
    lower_bound = -2*abs(knots_k[0])
    
    # Test boundaries
    
    # if knots_k[0] < lower_bound:
    #     print('Warning: lower bound violated')
        
    if knots_k[-1] > upper_bound:
        print('Warning: upper bound violated')
        
    # Dual codebook
    
    
    if optimized == True: 
        
        a = lower_bound
        b = upper_bound
        
        Gamma_k = []
        for jj in range(0, N_k - 2):
            midpoint = (knots_k[jj + 1] + knots_k[jj])/2
            Gamma_k.append(midpoint)
            
        Gamma_k.insert(0, a)
        Gamma_k.append(b)
    #      
    if optimized == False:
        a = 0
        b = upper_bound
        Gamma_k = [a]
        sstep = abs(b - a)/(N_k - 1)
        for _ in range(int((N_k - 2))):
            Gamma_k.append(Gamma_k[-1] + sstep)
        Gamma_k.append(b)

    return Gamma_k


# Dual map

In [5]:
def dual_map(eta, k, u_rv, alpha_list, N_list, prob_list, knots_lin, cell_bounds, optimized):
    
    N_k = N_list[k - 1]
    gamma_k = dual_grid(alpha_list, N_list, prob_list, k, knots_lin, cell_bounds, optimized)
    
    dual_map = 0
    
    for jj in range(0, N_k - 1):
        
        if 0 <= u_rv < (gamma_k[jj + 1] - eta)/(gamma_k[jj + 1] - gamma_k[jj]):
            w1 = 1
        else:
            w1 = 0
    
        if (gamma_k[jj + 1] - eta)/(gamma_k[jj + 1] - gamma_k[jj]) <= u_rv <= 1:
            w2 = 1
        else:
            w2 = 0
    
        if gamma_k[jj] <= eta < gamma_k[jj + 1]:
            w3 = 1
        else:
            w3 = 0
        
        dual_map = dual_map + (gamma_k[jj]*w1 + gamma_k[jj + 1]*w2)*w3
        #
    if eta == gamma_k[-1]:
        dual_map = dual_map + gamma_k[-1]
        
    return dual_map

# Probability function

In [6]:
# Function j^{*} in notes
def js(eta, k, alpha_list, N_list, prob_list, knots_lin, cell_bounds, optimized):
    
    N_k = N_list[k - 1]
    
    gamma_k = dual_grid(alpha_list, N_list, prob_list, k, knots_lin, cell_bounds, optimized)
        
    sum_j = 0
        
    for ll in range(0, N_k - 1):
            
            
        if  gamma_k[ll] <= eta < gamma_k[ll + 1]:
            w1 = 1
                
        else:
            w1 = 0
            
        sum_j = sum_j + w1*ll
            
            
    if eta == gamma_k[N_k - 1]:
        w2 = 1
            
    else:
        w2 = 0
            
    js = sum_j + w2*(N_k - 2)
        
    return js
    
# Function \lambda_{kl} from the paper
def lambda_kl(eta, k, l, alpha_list, N_list, prob_list, knots_lin, cell_bounds, optimized):
    
    N_k  = N_list[k - 1]
    gamma_k = dual_grid(alpha_list, N_list, prob_list, k, knots_lin, cell_bounds, optimized)
        
    if js(eta, k, alpha_list, N_list, prob_list, knots_lin, cell_bounds, optimized) == l:
            
        lambda_kl = (gamma_k[l + 1] - eta)/(gamma_k[l + 1] - gamma_k[l])
            
    elif  js(eta, k, alpha_list, N_list, prob_list, knots_lin, cell_bounds, optimized) == l - 1:
            
        lambda_kl = (eta - gamma_k[l - 1])/(gamma_k[l] - gamma_k[l - 1])
            
    else:
        lambda_kl = 0
        
    return lambda_kl


# This is the Eq. in point 4.2 (ii) of the notes

def dual_probability_l(P_prev_list, k, l, N_list, prob_list, alpha_list, optimized):
    
    N_k = N_list[k - 1]
    p_k = prob_list[k - 1]
    a_k = alpha_list[k - 1]
    
    gamma_k_minus_1 = dual_grid(alpha_list, N_list, prob_list, k - 1, knots_lin, cell_bounds, optimized)
    
    Pl = 0
    
    for jj in range(0, N_k):
        
        lambda1_kl = lambda_kl(gamma_k_minus_1[jj], k, l, alpha_list, N_list, prob_list, knots_lin, cell_bounds, optimized)
        lambda2_kl = lambda_kl(gamma_k_minus_1[jj] + a_k, k, l, alpha_list, N_list, prob_list, knots_lin, cell_bounds, optimized)
        
        Pl = Pl + ((1 - p_k)*lambda1_kl + p_k*lambda2_kl)*P_prev_list[jj]
        
    return Pl

# Define quantized average

In [7]:
def AverageQuantization(PQuantized, alpha_list, N_list, prob_list, n, function, par, knots_lin, cell_bounds, optimized):

    gamma_n = dual_grid(alpha_list, N_list, prob_list, n, knots_lin, cell_bounds, optimized)
    
    avg = 0
    for ll in range(0, N_list[-1]):
        avg += function(gamma_n[ll], par)*PQuantized[-1][ll]
        
    return avg

# Data results: save on file

In [8]:
from ast import literal_eval

def print_on_file(filename, result):
    
    with open(filename, 'a') as file:
        for el in result:
            file.write(str(el))
            file.write('\n')
    file.close()
    
    return file

def read_from_file(filename):
    
    result = []
    
    file = open(filename,'r')
    for el in file:
        result.append(literal_eval(el.strip()))
            
    file.close()
    
    return result

# Continuous probability

Analitically solvable result for comparison

In [9]:
from scipy import special
from math import factorial
from numpy import zeros

def EFunction(sigma, p0):

    return (p0/2)*(1 - erf( (log(p0) + ((sigma**2)/2) )/( sqrt(2)*sigma ) )) +\
    (1/2)*(1 + erf( (log(p0) - ((sigma**2)/2) )/( sqrt(2)*sigma ) ))

def ContinuousProbability(n, k, sigma, p0):
    
    EFun =  EFunction(sigma, p0)

    Cn = factorial(n)/( factorial(k)*factorial(n - k) )
    
    return Cn*(EFun**k)*((1 - EFun)**(n - k))


def ConditionalProbability(m, p_list):

    pi_list = zeros(m + 1)

    bin_num = []

    for ii in range(0, 2**m):
        bin_num.append(bin(ii)[2:].zfill(m))
        

    for tt in range(0, len(bin_num)):

        
        ib = bin_num[tt]
        
        factor = 1
        kpar = 0

        for jj in range(0, m):
            
            p_j = p_list[jj]
            ptilde_j = min(1, p_j)
                    
            if ib[jj] == '1':
                factor = factor*ptilde_j
                kpar += 1
                
            elif ib[jj] == '0':
                factor = factor*(1 - ptilde_j)
            
        pi_list[kpar] = pi_list[kpar] + factor

    return pi_list

# Average with conditional continuous probability

This is for comparison with an analitically solvable results for an homogeneus random walk

$E(G(X^n)) = \sum_k G(\alpha_0 k) P(X^n = \alpha_0 k)$

In [10]:
def AverageContinuos(PContinuous, alpha0, n, function, par):
    
    k_list = arange(1, n + 1, 1)
    
    GValues = []
    for kk in range(0, len(k_list)):
        tt = function(alpha0*k_list[kk], par)
        GValues.append(tt)
    
    AverageContinuous  = 0
    for kk in range(0, len(k_list)):
        AverageContinuous += GValues[kk]*PContinuous[kk]
    
    return AverageContinuous
    

def ConditionalAverageContinuous(alpha0, n, p_list, function, par):
    
    k_list = arange(0, n + 1, 1)
    
    GValues = []
    for kk in range(0, len(k_list)):
        tt = function(alpha0*k_list[kk], par)
        GValues.append(tt)

    
    bin_num = []

    for ii in range(0, 2**n):
        bin_num.append(bin(ii)[2:].zfill(n))

    cond_avg = 0   

    for tt in range(0, len(bin_num)):

        
        ib = bin_num[tt]
        
        factor = 1
        kpar = 0

        for jj in range(0, n):
            
            p_j = p_list[jj]
            ptilde_j = min(1, p_j)
                    
            if ib[jj] == '1':
                factor = factor*ptilde_j
                kpar += 1
                
            elif ib[jj] == '0':
                factor = factor*(1 - ptilde_j)
            
        cond_avg = cond_avg + GValues[kpar]*factor


    return cond_avg   

Function

In [11]:
def StrikePrice(X, strike):

    return max(0, X - strike)

# Simulation

Define lists of LogNormal probabilities so it is always the same for all calculations

In [12]:
p0 = 0.1
sigma0 = 0.5
nmax = 200
# prob_list_nmax = prob(p0, sigma0, nmax)
# print_on_file('LogNormalProb_nmax' + str(nmax) + '.txt', prob_list_nmax)


Define parameters

In [13]:
################# Define parameters ################
# Random walk
X_0 = 0
X_walk = [X_0]


n = 25 # total steps
alpha = 1

# In this case, all alphas are the same
alpha_list = [alpha]*n # alpha_k list
prob_list = read_from_file('LogNormalProb_nmax' + str(nmax) + '.txt')[:n] # p_k lists

# N_k list, or number of knots per level k
Nmax = 512
N_list = [Nmax]*n

ValOpt = False # Optimal if True, Arbitrary linear if false
# Create knots for corresponding N_k
qnt = optimal_quantizers(Nmax - 1)
qnt.create_linear_knots()
knots_lin = list(qnt.knots_linear)
cell_bounds = qnt.cell_bounds_linear


##############################################

# print('List of generated probabilities p_i = ' + str(prob_list))

Quantized simulation

In [15]:
from numpy import ones

for k in range(1, n + 1):
    
    a_k = alpha_list[k - 1]
    
    Z_k = random.binomial(1, prob_list[k - 1], size = 1)
    
    e_k = X_walk[-1] + a_k*Z_k
    
    u_k = random.uniform(0, 1)
    
    X_k = dual_map(e_k, k, u_k, alpha_list, N_list, prob_list, knots_lin, cell_bounds, ValOpt)

    X_walk.append(X_k)
    

# Probabilities

P_list = []
P0_list = []

for jj in range(0, N_list[0]):
    
    if jj == 0:
        P_0 = 1
        
    elif jj != 0:
        P_0 = 0
        
    P0_list.append(P_0)

P_list.append(P0_list)

for k in range(1, n + 1):
    
    print('k = '+ str(k))
    
    gamma_k = dual_grid(alpha_list, N_list, prob_list, k, knots_lin, cell_bounds, ValOpt)
    
    Pl_list = []
    
    for l in range(0, N_list[k - 1]):
        # print('k = '+ str(k) + ', l = ' + str(l) )
        P_l = dual_probability_l(P_list[-1], k, l, N_list, prob_list, alpha_list, ValOpt)
        # print('Pl = ' + str(P_l)) a
        Pl_list.append(P_l)
        
    P_list.append(Pl_list)
    
# print('Probabilities = ' + str(P_list))

print_on_file('ProbabilityQuant_n' + str(n) + '_N' + str(Nmax) + '.txt', P_list)

# Error function

In [16]:
def CDFContinuous(PContinuous, alpha0, ns, x_point):

    k_list = arange(0, ns + 1, 1)
    a = alpha0*k_list[0]
    b = alpha0*k_list[-1]
    # print('[a, b] = [' + str(a) + ', ' + str(b) + ']')


    cdf_x = 0
    for kk in range(0, len(k_list)):
        if alpha0*k_list[kk] < x_point:
            cdf_x = cdf_x + PContinuous[kk]

    return cdf_x
            

def CDFQuantization(PQuantized, alpha_list, N_list, prob_list, ns, knots_lin, cell_bounds, optimized, x_point):

    gamma_n = dual_grid(alpha_list, N_list, prob_list, ns, knots_lin, cell_bounds, optimized)
    
    # a = gamma_n[0]
    # b = gamma_n[-1]
    # print('[a, b] = [' + str(a) + ', ' + str(b) + ']')

    cdf_x = 0
    for ll in range(0, len(gamma_n)):
        if gamma_n[ll] < x_point:
            cdf_x = cdf_x + PQuantized[-1][ll]
        
    return cdf_x


def error(PContinuous, PQuantized, alpha_list, N_list, prob_list, ns, knots_lin, cell_bounds, optimized, a, b):
    
    alpha = alpha_list[0]

    ###
    xlist= arange(a, b, 1/1000)
    err = []
    for xx in xlist:
        err.append(abs(CDFContinuous(PContinuous, alpha, ns, xx) - \
                              CDFQuantization(PQuantized, alpha_list, N_list, prob_list, ns, knots_lin, cell_bounds, optimized, xx))**2)
        
    from scipy.integrate import simpson
    intg = simpson(err, x = xlist)

    # intg = quad(lambda xx: abs(CDFContinuous(PContinuous, alpha, n, xx) - \
    #                           CDFQuantization(PQuantized, alpha_list, N_list, prob_list, n, knots_lin, cell_bounds, optimized, xx))**2 ,\
    #            a, b, limit = 1000)[0]

    err2 = sqrt(intg)

    return err2
        

# An application: Strikeprice

In [18]:
strikes = [x for x in range(0, 31)]
StrikePrice_Cont = []
for K in strikes:
    print('K = ' + str(K))
    cond_avg_strike = ConditionalAverageContinuous(alpha, n, prob_list, StrikePrice, K)
    StrikePrice_Cont.append(cond_avg_strike)
# print_on_file('ProbabilityCont_n' + str(n) + '.txt', cond_prob)

In [21]:
strikes = [x for x in range(0, 31)]

Nnlist = [4, 8, 16, 32, 64, 128, 256, 512]

StrikePrice_Quant = [ [] ]*len(Nnlist)

for NN in range(0, len(Nnlist)):

    n = 25
    alpha_list = [alpha]*n # alpha_k list
    prob_list = read_from_file('LogNormalProb_nmax' + str(nmax) + '.txt')[:n] # p_k lists
    N_list = [Nnlist[NN]]*n

    ValOpt = False # Optimal if True, Arbitrary linear if false
    # Create knots for corresponding N_k
    qnt = optimal_quantizers(Nnlist[NN] - 1)
    qnt.create_linear_knots()
    knots_lin = list(qnt.knots_linear)
    cell_bounds = qnt.cell_bounds_linear

    probQ = read_from_file('prob_dual_grid/n' + str(n) + '/ProbabilityQuant_n' + str(n) +'_N' + str(Nnlist[NN]) + '.txt')

    for jj in range(0, len(strikes)):

        #print('K = ' + str(jj))
        qq = AverageQuantization(probQ, alpha_list, N_list, prob_list, n, StrikePrice, strikes[jj], knots_lin, cell_bounds, ValOpt)
        StrikePrice_Quant[NN] = append(StrikePrice_Quant[NN], qq)